In [3]:
import pandas as pd

df = pd.read_csv(
    "../../data/lc_small.csv",
    usecols=[
        "loan_status",
        "purpose",
        "home_ownership",
        "annual_inc",
        "verification_status",
        "dti",
    ],
)

df.dropna(inplace=True)  # Eliminate missing data
print(df.shape, "\n")
df.loan_status.value_counts()

(10466, 6) 



loan_status
Current               6602
Fully Paid            2722
Charged Off            898
Late (31-120 days)     152
In Grace Period         56
Late (16-30 days)       36
Name: count, dtype: int64

# Data Preparation

In [4]:
# Check for missing data

print(df.isna().sum(), "\n")



loan_status            0
purpose                0
home_ownership         0
annual_inc             0
verification_status    0
dti                    0
dtype: int64 



# Modeling

In [7]:
y = df.loan_status
X = df.drop("loan_status", axis=1)

X = pd.get_dummies(X, drop_first=True, dtype='int64')
X.head()

,annual_inc,dti,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified
0,62500.0,2.31,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,68000.0,16.03,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,150000.0,17.63,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,164000.0,19.51,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,122000.0,50.55,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
